In [2]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="osm",
    user="postgres",
    password="postgres")

cursor = conn.cursor()
cursor.execute("""
    SELECT poi.poi_id, poi.name, poi_coordinates.latitude, poi_coordinates.longitude, poi.district_id
    FROM poi 
    JOIN poi_coordinates ON poi.poi_id=poi_coordinates.poi_id
""")

# результат с данными
data = cursor.fetchall()

cursor.close()
conn.close()

In [3]:
import csv
with open('D:\\vkrb\\csv\\poi.csv', 'w', newline='', encoding='utf-8-sig') as file:
    record = csv.writer(file)
    record.writerow(['poi_id', 'name', 'latitude', 'longitude', 'district_id'])
    record.writerows(data)

In [4]:
import plotly.graph_objects as go
# бесплатный ключ доступа можно найти здесь https://account.mapbox.com
MAPBOX_ACCESS_TOKEN = "pk.eyJ1IjoiY3NpY3NhY3NvIiwiYSI6ImNsaWFpM3B2bzAzcTUzbXFwZ2ZjdnVpajEifQ.UY-B4Tg9KH0NXNC423X7Jg"

In [5]:
import pandas as pd
data = pd.read_csv('D:\\vkrb\\csv\\poi.csv')

In [10]:
import plotly.express as px

district_sum =data.groupby('district_id').size().reset_index(name='Плотность')

data_district_sum = pd.merge(data, district_sum, on='district_id')

px.set_mapbox_access_token(MAPBOX_ACCESS_TOKEN)
fig = px.scatter_mapbox(data_district_sum, lat='latitude', lon='longitude',
                        color='Плотность', size='Плотность',
                        width=1000, height=800, 
                        hover_name='name', # hover_data='district_id',
                        mapbox_style='open-street-map')
fig.show()